In [ ]:
! pip install ../build/azure_search_documents-11.4.0b4-py3-none-any.whl 
! pip install ../build/acslangchain-0.0.3-py3-none-any.whl 
! pip install openai
! pip install python-dotenv
! pip install numpy
! pip install pydantic
! pip install langchain

In [ ]:
# Import required libraries
import os, json
import openai
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings

from acslangchain.azuresearch import AzureSearch

In [ ]:
# Load environment variables
load_dotenv()

# Azure OpenAI settings
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
model: str = os.getenv('OPENAI_EMBEDDINGS_ENGINE_DOC', "text-embedding-ada-002")

In [ ]:
# Vector store settings
vector_store_address: str = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")
index_name: str = "embeddings"

In [ ]:
# Create Embeddings and Vector store
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=model, chunk_size=1) 
vector_store: AzureSearch = AzureSearch(azure_cognitive_search_name= vector_store_address, azure_cognitive_search_key= vector_store_password, index_name=index_name, embedding_function=embeddings.embed_query)    

In [ ]:
# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

max_input: int = 40
# Add texts to vector store
vector_store.add_texts(
    texts=list(map(lambda x: x['content'], input_data[0:max_input])),
    metadatas=list(map(lambda x: { 'title': x['title'], 'key': x['id'] }, input_data[0:max_input])),
    keys=list(map(lambda x: x['id'], input_data[0:max_input]))
)

In [ ]:
# Perform a similarity search
vector_store.similarity_search(query="What's Azure Cognitive Search?", k=3)